<a href="https://colab.research.google.com/github/kikiymini/7506R-1C2024-GRUPO02/blob/main/TP2/7506R_TP2_GRUPO2_Ensamble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Organizaci&oacute;n de Datos</center>

#### <center>C&aacute;tedra Ing. Rodriguez, Juan Manuel </center>

## <center>Trabajo Práctico 2: Críticas cinematográficas</center>

### <center> Grupo 2</center>

## Integrantes:

*   Aramayo Carolina
*   Utrera Maximo Damian
*   Villalba Ana Daniela
*   Fiorilo Roy


# Importación de librerias

In [11]:
from google.colab import drive

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Modelos
import joblib
from sklearn.ensemble import VotingClassifier

import warnings
warnings.filterwarnings("ignore")

RAND_SEED = 42

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


# Lectura de archivo


### Desde google drive

In [2]:
drive.mount('/content/drive')
root_path = "/content/drive/MyDrive/7506R-1C2024-GRUPO02/TP2/"

Mounted at /content/drive


### Desde maquina local

In [ ]:
# Si se esta trabajando en local, descomentar esta celta y comentar la de arriba
# root_path = "./"

In [3]:
train_path = root_path + 'Dataset/train.csv'
test_path = root_path + 'Dataset/test.csv'
sample_path = root_path + 'Dataset/sample_solution.csv'

ds_train = pd.read_csv(train_path)
ds_test = pd.read_csv(test_path)
sample = pd.read_csv(sample_path)

nb_path = root_path + 'Models/naive_bayes.pkl'
rf_path = root_path + 'Models/random_forest.pkl'
xgb_path = root_path + 'Models/xgboost.pkl'

nb = joblib.load(nb_path)
rf = joblib.load(rf_path)
xgb = joblib.load(xgb_path)

# Ensamble

### Utils

In [4]:
def map_sentiment(ds, invert=False):
  return ds.map({'negativo': 0, 'positivo': 1}) if not invert else ds.map({0: 'negativo', 1: 'positivo'})

In [5]:
def build_results(pred):
  res = ds_test.copy()
  res.drop(columns=["review_es"], inplace=True)
  res['sentimiento'] = pred
  return res

In [6]:
def print_positive_negatives(sentimientos):
  total_positives = (sentimientos == "positivo").sum()
  total_negatives = (sentimientos == "negativo").sum()
  print(f"Positives: {total_positives}\nNegatives: {total_negatives}")

In [16]:
x_test_text = ds_test["review_es"]
x_train_text = ds_train["review_es"]
y_train = map_sentiment(ds_train['sentimiento'])

## Vectorizacion

In [12]:
def clean_text(text):
  return text.strip()

def tokenizer(text):
  return [w for w in word_tokenize(text) if w.isalpha()]

stoplist = stopwords.words("spanish")

In [17]:
tfidf_vect = TfidfVectorizer(
    preprocessor=clean_text,
    tokenizer=tokenizer,
    min_df=5,
    stop_words=stoplist,
    ngram_range=(1, 2)
)
x_train = tfidf_vect.fit_transform(x_train_text)
x_test = tfidf_vect.transform(x_test_text)
x_train

<50000x189296 sparse matrix of type '<class 'numpy.float64'>'
	with 7169809 stored elements in Compressed Sparse Row format>

## Ensamblaje y votacion

In [18]:
ensemble_model = VotingClassifier(
    estimators=[
        ('mnb', nb),
        ('rf', rf),
        ('xgb', xgb)
    ],
    voting='hard'
)

ensemble_model.fit(x_train, y_train)

ValueError: could not convert string to float: 'La mayor virtud de esta película es su existencia.El hecho de que podamos jugar con los tópicos más extremos de las identidades patrias (la andaluza y la vasca) sin que nadie se escandalice ni ponga el grito en el cielo, indica mucho de nuestra madurez como nación (pese a quien pese). Bueno, corrijo: el hecho de que podamos jugar y hacer mofa y befa de los tópicos sobre los vascos y el nacionalismo vasco, sin que nadie se escandalice ni ponga el grito en el cielo, indica mucho del grado de normalización de ciertas cuestiones que antes eran llagas abiertas siempre dispuestas a sangrar. Y hago esta corrección, porque los andaluces han sido motivo de guasa siempre y nunca ha pasado nada.Por esto mismo, el planteamiento de "Ocho Apellidos Vascos" es valiente, es oportuno y es oportunista. Seguramente sea esa una de las principales razones por la que los españoles hemos acudido en masa, en una masa casi sin precedentes, a los cines a ver este 

In [19]:
# Predicciones
y_pred = ensemble_model.predict(x_test)
y_pred = map_sentiment(pd.Series(y_pred), invert=True)

In [20]:
resultados_ensamble = build_results(y_pred)
print_positive_negatives(resultados_ensamble['sentimiento'])

Positives: 5246
Negatives: 3353


# Guardado

In [21]:
# ensemble
resultados_ensamble.to_csv(root_path + "Dataset/resultados_ensamble.csv", index=False)
joblib.dump(ensemble_model, root_path + 'Models/ensamble.pkl')

['/content/drive/MyDrive/7506R-1C2024-GRUPO02/TP2/Models/ensamble.pkl']